**Implementing Regularization with Linear Regression**

In [2]:
import numpy as np
import pandas as pd
import zipfile
import requests
from io import BytesIO

# Define the URL of the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip'

# Download the ZIP file
response = requests.get(url)
zip_file = zipfile.ZipFile(BytesIO(response.content))

# Extract and read the specific CSV file
with zip_file.open('student-mat.csv') as file:
    df = pd.read_csv(file, sep=';')

# Save the DataFrame to a CSV file
df.to_csv('student_math.csv', index=False)

In [4]:
df = pd.read_csv('student_math.csv')
print(df.columns, df.shape)

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object') (395, 33)


In [6]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [8]:
df = pd.get_dummies(df)

In [17]:
#set our predictor and outcome variables and perform a train-test-split:
y = df['G3']
X = df.drop(columns = ['G3'])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [19]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder

# Convert categorical columns to one-hot encoding
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

# Ensure that both training and testing data have the same columns
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

# Fit the model
lasso = Lasso(alpha=0.05)
lasso.fit(X_train_encoded, y_train)


Lasso(alpha=0.05)

In [21]:
# Standardize the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
## an array of alpha values between 0.000001 and 1.0
alpha_array = np.logspace(-6, 0, 100)

#dict with key (alpha) and values being alpha_array
tuned_parameters = [{'alpha': alpha_array}]


In [27]:
#check the result of implementing different values of alpha 
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(estimator=Lasso(max_iter=20000), param_grid=tuned_parameters, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
model.fit(X,y)

/Users/Surface1/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.136e+00, tolerance: 7.172e-01
  model = cd_fast.enet_coordinate_descent(
/Users/Surface1/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.301e+00, tolerance: 5.921e-01
  model = cd_fast.enet_coordinate_descent(
/Users/Surface1/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

GridSearchCV(cv=5, estimator=Lasso(max_iter=20000),
             param_grid=[{'alpha': array([1.00000000e-06, 1.14975700e-06, 1.32194115e-06, 1.51991108e-06,
       1.74752840e-06, 2.00923300e-06, 2.31012970e-06, 2.65608778e-06,
       3.05385551e-06, 3.51119173e-06, 4.03701726e-06, 4.64158883e-06,
       5.33669923e-06, 6.13590727e-06, 7.05480231e-06, 8.11130831e-06,
       9.32603347e-06, 1.07226722e-05, 1....
       7.05480231e-02, 8.11130831e-02, 9.32603347e-02, 1.07226722e-01,
       1.23284674e-01, 1.41747416e-01, 1.62975083e-01, 1.87381742e-01,
       2.15443469e-01, 2.47707636e-01, 2.84803587e-01, 3.27454916e-01,
       3.76493581e-01, 4.32876128e-01, 4.97702356e-01, 5.72236766e-01,
       6.57933225e-01, 7.56463328e-01, 8.69749003e-01, 1.00000000e+00])}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [29]:
test_scores = model.cv_results_['mean_test_score']
train_scores = model.cv_results_['mean_train_score']

In [31]:
print(model.best_params_, model.best_score_)

{'alpha': 0.12328467394420659} -3.743294901559114
